In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from datetime import datetime
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
import re
from time import sleep

In [2]:
chrome_options = Options()
chrome_options.add_argument('--lang=en-US')
chrome_options.add_argument('--headless')
driver_service = Service(executable_path = 'chromedriver')
driver = webdriver.Chrome(service=driver_service, options=chrome_options)
driver.get('https://www.facebook.com/pythonprogramming.net')
sleep(5)

In [3]:
def class_name_format(class_name):
    return re.sub(' +', ' ', class_name).replace(" ", ".") 

In [4]:
def scrap_info(class_name, index = 0):
    try:
        class_name_formatted = class_name_format(class_name)
        class_name_element = driver.find_elements(By.CLASS_NAME, class_name_formatted)[index]
        info = class_name_element.text
    except Exception as e:
        return f"A PROBLEM OCCURED {e} "
    return info     

In [14]:
def scrap_fb_python_page():    
    data = dict()  
    sleep(2)
    data["page_name"] = scrap_info('x78zum5 xdt5ytf x1wsgfga x9otpla')
    text_likes = scrap_info('x1i10hfl xjbqb8w x6umtig x1b1mbwd xaqea5y xav7gou x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz xt0b8zv xi81zsa x1s688f')
    data["nb_likes"] = text_likes.split(" ")[0]
    text_follower = scrap_info('x1i10hfl xjbqb8w x6umtig x1b1mbwd xaqea5y xav7gou x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz xt0b8zv xi81zsa x1s688f', 1)
    data["nb_follower"] = text_follower.split(" ")[0]
    text_page_type = scrap_info('x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x6prxxf xvq8zen xo1l8bm xzsf02u', 2)
    data["page_type"] = text_page_type.split("·")[1].strip()
    data["contact"]= scrap_info('x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x xudqn12 x3x7a5m x6prxxf xvq8zen xo1l8bm xzsf02u x1yc453h', 1)
    data["website"]= scrap_info('x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x xudqn12 x3x7a5m x6prxxf xvq8zen xo1l8bm x1qq9wsj x1yc453h')
    data["description"]= scrap_info('x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x6prxxf xvq8zen xo1l8bm xzsf02u',1)
    driver.get("https://www.facebook.com/pythonprogramming.net/about_profile_transparency")
    sleep(3)
    i = 0
    while i<3:
        try:
            data["creation_date"] = scrap_info('x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x xudqn12 x3x7a5m x6prxxf xvq8zen xo1l8bm xzsf02u',i)
            date_format = '%B %d, %Y'
            data["creation_date"] = datetime.strptime(data["creation_date"], date_format)
        except:
            i+=1
        else:
            break        
    driver.get('https://www.facebook.com/pythonprogramming.net')        
    return data             

In [15]:
data = scrap_fb_python_page()

In [16]:
data

{'page_name': 'Python Programming',
 'nb_likes': '152K',
 'nb_follower': '163K',
 'page_type': 'Interest',
 'contact': 'harrison@pythonprogramming.net',
 'website': 'pythonprogramming.net',
 'description': 'PythonProgramming.net is a programming tutorials / educational site containing over a thousand video',
 'creation_date': datetime.datetime(2015, 1, 10, 0, 0)}

# Save to mongodb

In [18]:
from pymongo import MongoClient

C:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [19]:
client = MongoClient("mongodb://localhost:27017/")
db = client["fc_data"]
collection = db["python_page"]

In [26]:
if "Python Programming" not in list(collection.find()):
    print("5")

5


In [27]:
list(collection.find())

[{'_id': ObjectId('63efe3fa77f146544df71c27'),
  'page_name': 'Python Programming',
  'nb_likes': '152K',
  'nb_follower': '163K',
  'page_type': 'Interest',
  'contact': 'harrison@pythonprogramming.net',
  'website': 'pythonprogramming.net',
  'description': 'PythonProgramming.net is a programming tutorials / educational site containing over a thousand video',
  'creation_date': datetime.datetime(2015, 1, 10, 0, 0)}]

In [ ]:
result = collection.insert_one(data)